<a href="https://colab.research.google.com/github/kenCompSciFox/Portfolio_Rebalancing/blob/master/project_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio Rebalancing Using Constraint Satisfaction
## CS510 Final Project


In [0]:
!pip install python-constraint

  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24080 sha256=b38aabfed2a2d98d6f05021a6fee9fddf4cac5b61b8dad37fd31dcc09bf5915f
  Stored in directory: /root/.cache/pip/wheels/34/31/15/7b070b25d0a549d20ce2e9fe6d727471c2c61ef904720fd40c
Successfully built python-constraint


In [0]:
from collections import namedtuple
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from constraint import Problem
from constraint import MaxSumConstraint
from constraint import MinSumConstraint

#### 1. Create a common stock only model based on the constituents and weighting of the Dow Jones Industrial Average (DJIA)

__Source__: https://www.slickcharts.com/dowjones

In [0]:
def try_float(f):
    """Tries to convert argument f to float.  Returns f as float if convertible, otherwise returns 
    original value of f"""
    try:
        return float(f)
    except:
        return f
    
def djia(lower_drift=2, upper_drift=2):
    """ Builds a model based on the Dow Jones Industrial Average"""
    
    djia_data_url = 'https://www.slickcharts.com/dowjones'
    response = requests.get(djia_data_url)
    if not response.ok:
        raise RuntimeError('Unable to retrieve page from https://www.slickcharts.com/dowjones')
        
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup('table')
    header = soup('thead')

    headers = [e.text.strip() for e in soup.find('table').find('thead').find('tr').find_all('th')]
    Constituent = namedtuple('Constituent', headers[1:-2])
    constituents = [Constituent(*[try_float(td.text.strip()) 
                       for td in e.find_all('td')[1:-2]]) 
                          for e in soup.find('table').find('tbody').find_all('tr')]
    model = pd.DataFrame(constituents)
    model['Lower_Drift'] = lower_drift
    model['Upper_Drift'] = upper_drift
    model['Index'] = model.Symbol
    
    return model.set_index('Index')

model = djia()
display(model[:5])

,Company,Symbol,Weight,Price,Lower_Drift,Upper_Drift
Index,,,,,,
BA,Boeing Company,BA,8.941669,371.42,2,2
UNH,UnitedHealth Group Incorporated,UNH,6.748222,276.01,2,2
AAPL,Apple Inc.,AAPL,6.393426,261.79,2,2
HD,Home Depot Inc.,HD,5.332694,217.70,2,2
GS,Goldman Sachs Group Inc.,GS,5.325130,220.30,2,2


#### 2. Create Holdings and Restrictions

In [0]:
Holding = namedtuple('Holding', ['Symbol', 'Units', 'Price', 'Restricted'])

def create_cash_only_holdings(amount):
    """Creates a holdings dataframe with cash in the amount specified """
    
    holdings = pd.DataFrame([Holding('CASH', amount, 1, False)])
    holdings['Index'] = holdings.Symbol
    return holdings.set_index('Index')

def create_cash_only_holdings_with_restrictions(amount, restricted_symbols):
    holding_list = [Holding('CASH', amount, 1, False)]
    holding_list.extend([Holding(rs, 0, None, True) for rs in restricted_symbols])
    holdings = pd.DataFrame(holding_list)
    holdings['Index'] = holdings.Symbol
    return holdings.set_index('Index')
    
holdings = create_cash_only_holdings_with_restrictions(10_000_000, ['AAPL', 'BA']) 
display(holdings)

,Symbol,Units,Price,Restricted
Index,,,,
CASH,CASH,10000000,1.0,False
AAPL,AAPL,0,NaN,True
BA,BA,0,NaN,True


#### 3. Create a portfolio from a cash only holding based on the DJIA model


In [0]:
def create_portfolio(model, holdings, lower_drift = 0, upper_drift = 2):
    """ Creates a portfolio from a model dataframe and a holdings dataframe """
    
    # Perform an outer join on the two based on Symbol
    portfolio = model.join(holdings, how='outer', rsuffix='_Holdings')
    
    # Replace Symbol with Holdings Symbol if Symbol is Nan, then drop Symbol Holdings column
    portfolio.Symbol = [sh if pd.isnull(s) else s for s, sh in zip(portfolio.Symbol, portfolio.Symbol_Holdings)]
    portfolio.drop(columns='Symbol_Holdings', inplace=True)

    # Replace Company name with Symbol if Company Name is null
    portfolio.Company = [s if pd.isnull(c) else c for c, s in zip(portfolio.Company, portfolio.Symbol)]
    
    # Replace Price with Holdings Price if Price is Nan, then drop Price Holdings column
    portfolio.Price = [ph if pd.isnull(p) else p for p, ph in zip(portfolio.Price, portfolio.Price_Holdings)]
    portfolio.drop(columns='Price_Holdings', inplace=True)
    
    # Fill NaNs
    portfolio.fillna(value={'Weight': 0, 'Lower_Drift': lower_drift, 'Upper_Drift': upper_drift, 
                                        'Units': 0, 'Restricted': False}, inplace=True)

    # Calculate market value
    portfolio['Market_Value'] = portfolio.Price * portfolio.Units
    total_market_value = sum(portfolio.Market_Value)
    
    #Adjust targets.  This consists of two pieces: 1. Freeze targets to actual for restricted securities and
    #   2. Prorate the remaining to add up to 100%
    
    portfolio['Target'] = [mv/total_market_value if r else w 
                               for mv, w, r in zip(portfolio.Market_Value, portfolio.Weight, portfolio.Restricted)]
    sum_of_frozen_targets = sum(t for t in portfolio[portfolio.Restricted].Target)
    sum_of_targets = portfolio.Target.sum()
    sum_of_unfrozen_targets = sum_of_targets - sum_of_frozen_targets
    #portfolio.Target = portfolio.Target/sum_of_targets * 100
    portfolio['Target'] = [t if r else t/sum_of_unfrozen_targets * (100 - sum_of_frozen_targets) 
                               for t, r in zip(portfolio.Target, portfolio.Restricted)]
    
    # Calculate market value and related amounts in order to perform and validate rebalance
    
    portfolio['Target_Market_Value'] = portfolio.Target/100 * total_market_value
    portfolio['Min_Market_Value'] = (portfolio.Weight - portfolio.Lower_Drift)/100 * total_market_value
    portfolio.Min_Market_Value = [max(mmm, 0) for mmm in portfolio.Min_Market_Value]
    portfolio['Max_Market_Value'] = (portfolio.Target + portfolio.Upper_Drift)/100 * total_market_value
    portfolio['Min_Units_To_Transact'] = (portfolio.Min_Market_Value - portfolio.Market_Value)/portfolio.Price
    portfolio.Min_Units_To_Transact = [math.ceil(u) for u in portfolio.Min_Units_To_Transact]
    portfolio['Max_Units_To_Transact'] = (portfolio.Max_Market_Value - portfolio.Market_Value)/portfolio.Price
    portfolio.Max_Units_To_Transact = [math.ceil(u) for u in portfolio.Max_Units_To_Transact]
    
    return portfolio
    
portfolio = create_portfolio(model, holdings)    
display(portfolio[:5])  


,Company,Symbol,Weight,Price,Lower_Drift,Upper_Drift,Units,Restricted,Market_Value,Target,Target_Market_Value,Min_Market_Value,Max_Market_Value,Min_Units_To_Transact,Max_Units_To_Transact
Index,,,,,,,,,,,,,,,
AAPL,Apple Inc.,AAPL,6.393426,261.79,2.0,2.0,0.0,True,0.0,0.000000,0.000000,439342.6,200000.000000,1679,764
AXP,American Express Company,AXP,2.898893,119.06,2.0,2.0,0.0,False,0.0,3.428007,342800.737866,89889.3,542800.737866,755,4560
BA,Boeing Company,BA,8.941669,371.42,2.0,2.0,0.0,True,0.0,0.000000,0.000000,694166.9,200000.000000,1869,539
CASH,CASH,CASH,0.000000,1.00,0.0,2.0,10000000.0,False,10000000.0,0.000000,0.000000,0.0,200000.000000,-10000000,-9800000
CAT,Caterpillar Inc.,CAT,3.498193,143.88,2.0,2.0,0.0,False,0.0,4.136693,413669.335708,149819.3,613669.335708,1042,4266


#### 4. Rebalance the Portfolio

In [0]:
class NoSolutionFound(Exception):
    pass


def rebalance(portfolio, max_drift=0.0001, increment=0.0001, verbose=False):
    """ Rebalance a portfolio dataframe.  This method starts with a very narrow drift range then increases up to the maximum
    drift range until a solution is found.  Returns the first valid solution.  If no solution is found,
    raises a runtime error. """
    
    problem = Problem()

    min_cash = max_cash = 0
    prices = []

    try:
        at_limit = 0
        non_cash_positions = 0
        for position in portfolio.itertuples():
            if position.Symbol == 'CASH':
                min_cash, max_cash = -position.Max_Units_To_Transact, -position.Min_Units_To_Transact
            else:
                if position.Restricted:
                    continue
                non_cash_positions += 1
                if position.Min_Units_To_Transact != position.Max_Units_To_Transact:
                    min_units = int((position.Target_Market_Value - position.Target_Market_Value * max_drift)//position.Price)
                    max_units = int((position.Target_Market_Value + position.Target_Market_Value * max_drift)//position.Price)
                    
                    if min_units < position.Min_Units_To_Transact and max_units > position.Max_Units_To_Transact:
                        at_limit += 1

                    domain = [d for d in range(max(position.Min_Units_To_Transact, min_units), min(position.Max_Units_To_Transact, max_units)) ]
                    if not domain: 
                        raise NoSolutionFound()
                elif position.Min_Units_To_Transact == position.Max_Units_To_Transact:
                    domain = [position.Min_Units_To_Transact]
                elif position.Min_Units_To_Transact > position.Max_Units_To_Transact:
                    domain = [0]
                problem.addVariable(position.Symbol, domain)
                
                if verbose:
                    print(position.Symbol, position.Price, domain)
                
                prices.append(position.Price)

        problem.addConstraint(MaxSumConstraint(max_cash, prices))   
        problem.addConstraint(MinSumConstraint(min_cash, prices))   

        solution = problem.getSolution()

        if solution:
            buy_units = []
            for symbol in list(portfolio.Symbol):
                try:
                    buy_units.append(solution[symbol])
                except:
                    buy_units.append(0)
        else:
            if at_limit == non_cash_positions:
                raise RuntimeError('No solution exists')

        portfolio['Buy_Units'] = buy_units
        portfolio['Proposed_Market_Value'] = (portfolio.Units + portfolio.Buy_Units) * portfolio.Price
        cash_mv = portfolio.loc[portfolio.Symbol =='CASH', 'Proposed_Market_Value']
        cash_buy_units = cash_mv - sum(p for p in portfolio.Proposed_Market_Value if not pd.isna(p))  # sum includes cash_mv
        portfolio.loc[portfolio.Symbol == 'CASH', 'Buy_Units'] =  cash_buy_units
        portfolio.loc[portfolio.Symbol =='CASH', 'Proposed_Market_Value'] = cash_mv + cash_buy_units
        portfolio['Drift_MV'] = portfolio.Target_Market_Value - portfolio.Proposed_Market_Value
        portfolio['Drift_Pct'] = portfolio.Drift_MV/portfolio.Target_Market_Value * 100
        return portfolio
        
    except NoSolutionFound:
        return rebalance(portfolio, max_drift=max_drift+increment, verbose=verbose)
    except Exception as e:
        raise(e)



%time rebalance(portfolio)
portfolio[:5]


CPU times: user 22.8 ms, sys: 954 µs, total: 23.7 ms
Wall time: 24.7 ms


,Company,Symbol,Weight,Price,Lower_Drift,Upper_Drift,Units,Restricted,Market_Value,Target,Target_Market_Value,Min_Market_Value,Max_Market_Value,Min_Units_To_Transact,Max_Units_To_Transact,Buy_Units,Proposed_Market_Value,Drift_MV,Drift_Pct
Index,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc.,AAPL,6.393426,261.79,2.0,2.0,0.0,True,0.0,0.000000,0.000000,439342.6,200000.000000,1679,764,0.00,0.00,0.000000,NaN
AXP,American Express Company,AXP,2.898893,119.06,2.0,2.0,0.0,False,0.0,3.428007,342800.737866,89889.3,542800.737866,755,4560,2878.00,342654.68,146.057866,0.042607
BA,Boeing Company,BA,8.941669,371.42,2.0,2.0,0.0,True,0.0,0.000000,0.000000,694166.9,200000.000000,1869,539,0.00,0.00,0.000000,NaN
CASH,CASH,CASH,0.000000,1.00,0.0,2.0,10000000.0,False,10000000.0,0.000000,0.000000,0.0,200000.000000,-10000000,-9800000,-9996489.12,3510.88,-3510.880000,-inf
CAT,Caterpillar Inc.,CAT,3.498193,143.88,2.0,2.0,0.0,False,0.0,4.136693,413669.335708,149819.3,613669.335708,1042,4266,2874.00,413511.12,158.215708,0.038247
